# Construct school interaction networks with increased between-class contacts

In [1]:
import networkx as nx
import pandas as pd
from os.path import join

# network construction utilities
import construct_school_network as csn

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

In this script, contact networks of "average" Austrian schools, depending on school type are created. These characteristics (mean number of classes, mean students per class) were determined from [statistics about Austrian schools](https://www.bmbwf.gv.at/Themen/schule/schulsystem/gd.html) (year 2017/18, page 10) and confirmed in interviews with a range of Austrian teachers and school directors conducted in December 2020. The school types modeled here are
* Primary schools (Volksschule), ```primary```
* Primary schools with daycare (Volksschule mit Ganztagesbetreuung), ```primary_dc```
* Lower secondary schools (Unterstufe), ```lower_secondary```
* Lower secondary schools with daycare (Unterstufe mit Ganztagesbetreuung), ```lower_secondary_dc```
* Upper secondary schools (Oberstufe), ```upper_secondary```
* Secondary schools (Gymnasium), ```secondary```
* Secondary schools with daycare (Gymnasium mit Ganztagesbetreuung), ```secondary_dc```  

For every school type, one network is created.

**NOTE**: A more detailed description about the design decisions entering the modeling of each school type can be found in the document ```school_type_documentation```. In the following, "students" always refers to the number of students per class.  

## Background information

### School characteristics

Descriptive school statistics are taken from [statistics](https://www.bmbwf.gv.at/Themen/schule/schulsystem/gd.html) about Austrian schools from 2017/18 and from a series of stakeholder-interviews with Austrian teachers and school directors conducted in December 2020.

In [2]:
# different age structures in Austrian school types
age_brackets = {'primary':[6, 7, 8, 9],
                'primary_dc':[6, 7, 8, 9],
                'lower_secondary':[10, 11, 12, 13],
                'lower_secondary_dc':[10, 11, 12, 13],
                'upper_secondary':[14, 15, 16, 17],
                'secondary':[10, 11, 12, 13, 14, 15, 16, 17],
                'secondary_dc':[10, 11, 12, 13, 14, 15, 16, 17]
               }

In [3]:
# average number of classes per school type and students per class
school_characteristics = {
    # Primary schools
    # Volksschule: schools 3033, classes: 18245, students: 339382
    'primary':            {'classes':8, 'students':19},
    'primary_dc':         {'classes':8, 'students':19},
    
    # Lower secondary schools
    # Hauptschule: schools 47, classes 104, students: 1993
    # Mittelschule: schools 1131, classes: 10354, students: 205905
    # Sonderschule: schools 292, classes: 1626, students: 14815
    # Total: schools: 1470, classes: 12084, students: 222713
    'lower_secondary':    {'classes':8, 'students':18},
    'lower_secondary_dc': {'classes':8, 'students':18},
    
    # Upper secondary schools
    # Oberstufenrealgymnasium: schools 114, classes 1183, students: 26211
    # BMHS: schools 734, classes 8042, students 187592
    # Total: schools: 848, classes 9225, students: 213803
    'upper_secondary':    {'classes':10, 'students':23}, # rounded down from 10.8 classes
    
    # Secondary schools
    # AHS Langform: schools 281, classes 7610, students 179633
    'secondary':          {'classes':28, 'students':24}, # rounded up from 27.1 classes
    'secondary_dc':       {'classes':28, 'students':24} # rounded up from 27.1 classes
}

### Characteristics of Austrian families

Family sizes with children < 18 years old from the [Austrian microcensus 2019](https://www.statistik.at/web_de/statistiken/menschen_und_gesellschaft/bevoelkerung/haushalte_familien_lebensformen/familien/index.html) (Note: 63.45 % of all households have no children), file ```familien_nach_familientyp_und_zahl_der_kinder_ausgewaehlter_altersgruppen_```:

* 1 child: 48.15 % (81.95 % two parents, 18.05 % single parents)
* 2 children: 38.12 % (89.70 % two parents, 10.30% single parents)
* 3 children: 10.69 % (88.26 % two parents, 11.74 % single parents)
* 4 or more children: 3.04 % (87.44 % two parents, 12.56 % single parents)

In [4]:
# given the precondition that the family has at least one child, how many
# children does the family have?
p_children = {1:0.4815, 2:0.3812, 3:0.1069, 4:0.0304}

# probability of being a single parent, depending on the number of children
p_parents = {1:{1:0.1805, 2:0.8195},
             2:{1:0.1030, 2:0.8970},
             3:{1:0.1174, 2:0.8826},
             4:{1:0.1256, 2:0.8744}
            }

General household sizes of households with one family (2.51% of households have more than one family) [Austrain household statistics 2019](https://www.statistik.at/web_de/statistiken/menschen_und_gesellschaft/bevoelkerung/haushalte_familien_lebensformen/haushalte/index.html), files 
* ```ergebnisse_im_ueberblick_privathaushalte_1985_-_2019```
* ```familien_nach_familientyp_und_zahl_der_kinder_ausgewaehlter_altersgruppen_``` 

Percentages:
* single $\frac{(3950 - 2388)}{3959}$ = 39.54 %
* couple, no kids $\frac{1001}{3959}$ = 25.28 % 
* single parent with one kid < 18: $\frac{277}{3950} \cdot \frac{87.0}{137.4}$ = 4.44 %
* single parent with two kids < 18: $\frac{277}{3950} \cdot \frac{37.3}{137.4}$ = 1.9%
* single parent with three or more kids < 18: $\frac{277}{3950} \cdot \frac{13.1}{137.4}$ = 0.67%
* couples with one kid < 18: $\frac{1050}{3950} \cdot \frac{252.4}{606.7}$ = 11.06 %
* couples with two kids < 18: $\frac{1050}{3950} \cdot \frac{255.5}{606.7}$ = 11.19 %
* couples with three or more kids <18: $\frac{1050}{3950} \cdot \frac{98.9}{606.7}$ = 4.33 % 
* households with three adults (statistic: household with  kids > 18 years): 1.59 % 

In [5]:
# probability of a household having a certain size, independent of having a child
teacher_p_adults = {1:0.4655, 2:0.5186, 3:0.0159}
teacher_p_children = {1:{0:0.8495, 1:0.0953, 2:0.0408, 3:0.0144},
                      2:{0:0.4874, 1:0.2133, 2:0.2158, 3:0.0835},
                      3:{0:1, 1:0, 2:0, 3:0}}

### Link type <-> contact type mapping

The simulation relies on specified contact strengths (close, intermediate, far, very far) to determine infection risk. Nevertheless, depending on the setting, there are a multitude of different contacts (link types) between different agent groups and during different activities. The below dictionary provides a complete list of all link types that exist in the school setting, and a mapping of every link type to the corresponding contact type.

In [6]:
contact_map = {
    'student_household':'close', 
    'student_student_intra_class':'far',
    'student_student_table_neighbour':'intermediate',
    'student_student_daycare':'far',
    'student_student_friends':'intermediate',
    'teacher_household':'close',
    'teacher_teacher_short':'far', 
    'teacher_teacher_long':'intermediate',
    'teacher_teacher_team_teaching':'intermediate',
    'teacher_teacher_daycare_supervision':'intermediate',
    'teaching_teacher_student':'intermediate',
    'daycare_supervision_teacher_student':'intermediate'
}
# Note: student_student_daycare overwrites student_student_intra_class and
# student_student_table_neighbour

# Note: teacher_teacher_daycare_supervision and teacher_teacher_team_teaching 
# overwrite teacher_teacher_short and teacher_teacher_long

### Teacher social contacts

Network density scores from an [article about interactions between teachers](https://academic.oup.com/her/article/23/1/62/834723?login=true) for "socialize with outside of school" (```r_friend```) and "engage in conversation regularly" (```r_conversation```).

In [7]:
r_teacher_friend = 0.059
r_teacher_conversation = 0.255

## Compose representative schools

In [9]:
dst = '../data/school/representative_schools_added_friend_contacts'
# in principle there is functionality in place to generate contacts
# between students in different classes, depending on the floor the
# classes are on. We currently don't use this functionality, as 
# schools all implement measures to keep between-class-contacts to
# a minimum- Therefore floor specifications are not important for our
# school layout and we just assume that all classes are on the same
# floor.
N_floors = 1
half = True

school_types = ['primary', 'primary_dc', 'lower_secondary','lower_secondary_dc',
                'upper_secondary', 'secondary']

for school_type in school_types:
    print(school_type)
    
    for friends_ratio in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]:
        print(friends_ratio)
        N_classes = school_characteristics[school_type]['classes']
        class_size = school_characteristics[school_type]['students']
        school_name = '{}_classes-{}_students-{}'.format(school_type,\
            N_classes, class_size)

        # generate the contact graph given all the information about the
        # school layout, household characteristics and contact character-
        # istics of teachers
        G, teacher_schedule, student_schedule = csn.compose_school_graph(\
                school_type, N_classes, class_size, N_floors, p_children,
                p_parents, teacher_p_adults, teacher_p_children, 
                r_teacher_conversation, r_teacher_friend)

        csn.add_between_class_contacts(friends_ratio, class_size, N_classes, G)
        
        # map the link types to contact types
        csn.map_contacts(G, contact_map)
        # for the interactive visualization, we also need a list of all
        # agents (nodes) in the contact graph and their attributes
        node_list = csn.get_node_list(G)
        node_list.to_csv(join(dst,'{}_node_list.csv'.format(school_name)),
                              index=False)

        # save the graph
        nx.readwrite.gpickle.write_gpickle(G, \
            join(dst,'{}_friends-{}_network.bz2'\
                 .format(school_name, friends_ratio)), protocol=4)

        # for the interactive visualization, we also need the respective
        # schedules of students and teachers for teaching days (i.e. non-
        # weekends)
        for schedule, agent_type in zip([teacher_schedule, student_schedule],
                                        ['teachers', 'students']):
            schedule.to_csv(join(dst,'{}_schedule_friends-{}_{}.csv'
                    .format(school_name, friends_ratio, agent_type)))
        
        if half:
            # if classes are halved as a prevention measure (i.e. only half the
            # students of a class come to school on any given day and the halves
            # alternate every day), we need to modify the edges in the contact
            # graph accordingly.
            csn.make_half_classes(class_size, N_classes, G, student_schedule)
            nx.readwrite.gpickle.write_gpickle(G, \
                join(dst,'{}_friends-{}_network_half.bz2'\
                        .format(school_name, friends_ratio)), protocol=4)
            for schedule, agent_type in zip([teacher_schedule, student_schedule],
                                        ['teachers', 'students']):
                schedule.to_csv(join(dst,'{}_schedule_friends-{}_{}_half.csv'
                        .format(school_name, friends_ratio, agent_type)))

primary
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
primary_dc
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
lower_secondary
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
lower_secondary_dc
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
upper_secondary
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
secondary
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
